In [3]:
import numpy as np
import pdb

In [74]:
def mape(X, predictions, y):
  num_timeslots = 43
  num_districts = 66
  if len(y.shape) == 1:
    y = np.asmatrix(y)
  if len(predictions.shape) == 1:
    predictions = np.asmatrix(predictions)
  Xy = np.concatenate((X, y.T, predictions.T), axis=1)
  district_scores = np.array([])
  for key in np.unique(X[:,0]):
    group = np.compress((Xy[:,0] == key).flat, Xy, axis=0)
    pdb.set_trace()
    district_scores[len(district_scores)] = (
      np.sum(
        (group[:,-2] -
         group[:,-1])/
        group[:,-2]
      ) / num_timeslots
    )
  return np.sum(district_scores) / num_districts

def mape_scorer(estimator, X, y):
  estimator.fit(X)
  predictions = estimator.predict(X)
  return mape(X, predictions, y)

Testing MAPE

In [ ]:
from sklearn.linear_model import LogisticRegression

est = LogisticRegression()
X = np.array([[1, 1], [1, 2], [2, 3], [2, 4]])
predictions = np.array([1, 2, 3, 4])
y = np.array([1, 2, 3, 4])

# Should return 0
mape(X, predictions, y)

> <ipython-input-74-99c0cfc38c45>(14)mape()
-> np.sum(
(Pdb) group[:-2]
matrix([], shape=(0, 4), dtype=int64)
(Pdb) group
matrix([[1, 1, 1, 1],
        [1, 2, 2, 2]])
(Pdb) group[0]
matrix([[1, 1, 1, 1]])
(Pdb) group[:]
matrix([[1, 1, 1, 1],
        [1, 2, 2, 2]])
(Pdb) group[:,2]
matrix([[1],
        [2]])
(Pdb) group[:,-2]
matrix([[1],
        [2]])
(Pdb) (group[:,-2]-group[:,-1])/group[:,-2]
matrix([[0],
        [0]])


In [63]:
np.where(Xy[:,0] == 2)

(array([2, 3]), array([0, 0]))

In [70]:
np.compress((Xy[:,0] == 2).flat, Xy, axis=0)[:,-2]

matrix([[3],
        [4]])

In [66]:
a[a[:,0]==1]
Xy[Xy[:,0] == 2]



matrix([[2, 2]])

In [45]:
a = np.asmatrix(y)
b = np.asmatrix(predictions)
Xy = np.concatenate((X, a.T, b.T), axis=1)
Xy

matrix([[1, 1, 1, 1],
        [1, 2, 2, 2],
        [2, 3, 3, 3],
        [2, 4, 4, 4]])

In [ ]:
%%sql --module q_all

SELECT * FROM [datalab-projects-1331:xjk_algo_comp.gaps_processed]
WHERE gap > 0
ORDER BY uuid()

In [ ]:
features_list = ['']